In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def getGraphForRun(s):
    for i in s.split("\n"):
        print("'"+i+"'", end = " ")
    return s.split("\n")

In [3]:
graphs =  """graph7
graph8.txt
fat200
GSE1730_q
GSE10158_q
orani
ego-facebook
grqc_q
uc64
government
wiki-elec
lastfm
hepPh
astroPh
condmat
Enron_q
fb-pages
brightkite
livemocha
gowalla
citeseer
com-dblp
amazon_q
youtube_q
hyves_q
skitter
flixster
patent_q
soc-livejournal
dblp-author"""

In [4]:
len(graphs.split("\n"))

30

In [5]:
graph_names = getGraphForRun(graphs)

'graph7' 'graph8.txt' 'fat200' 'GSE1730_q' 'GSE10158_q' 'orani' 'ego-facebook' 'grqc_q' 'uc64' 'government' 'wiki-elec' 'lastfm' 'hepPh' 'astroPh' 'condmat' 'Enron_q' 'fb-pages' 'brightkite' 'livemocha' 'gowalla' 'citeseer' 'com-dblp' 'amazon_q' 'youtube_q' 'hyves_q' 'skitter' 'flixster' 'patent_q' 'soc-livejournal' 'dblp-author' 

# CPU Experiment

In [7]:
def CPUArgs(path,graph_names):
    df = pd.read_csv(path)
    qid = []
    for graph_name in graph_names:
        l = []
        num_vertex = df[df['file_name']==graph_name]['V'].values[0]
        for core_value in [6,9,12,15,18,21,24]:
            vert = np.random.randint(0,num_vertex)
            l.append(vert)
        qid.append(" ".join(list(map(str,l))))
    return qid

In [8]:
cpu_args = CPUArgs("../../graphStatisticsFinal.csv",graph_names)

In [9]:
cpu_args

['73730 142944 94629 165694 170215 157030 100578',
 '168659 136316 36027 116835 63253 42457 19086',
 '170410 54697 183110 262990 278178 279803 168804',
 '174852 313830 82150 220506 78658 274848 312597',
 '673421 295426 857898 316803 139665 918619 880160',
 '941641 1341842 968483 174369 1042925 298843 260848',
 '117120 1069949 1422676 1505935 1665532 1517341 835452',
 '1236070 1994147 2272307 1967706 1123845 395257 1724213',
 '1154710 420314 2999380 1672218 2585736 1881330 3630921',
 '821868 2539355 2470345 3303316 3801386 2294618 604277',
 '120522 2431199 5765351 2288264 5370942 823817 856045']

# Experiment 1-2

In [ ]:
def GpuClientArgs(path,graph_names,output_dir,postfix="1 1"):
    df = pd.read_csv(path)
    for graph_name in graph_names:
        query_data = ''
        num_vertex = df[df['file_name']==graph_name]['V'].values[0]
        for N1,N2 in zip([3,6,9,12,15,18,21],[6,9,12,15,18,21,24]):
            qid = np.random.randint(0,num_vertex)
            query = f"{N1} {N2} {qid} {postfix}\n"
            query_data +=query
        query_data+="server_exit"
        with open(os.path.join(output_dir,graph_name),'w') as f:
            f.write(query_data)
        print("completed: ",graph_name)
    print("Done yay! :)")
x = GpuClientArgs("../../graphStatisticsFinal.csv",graph_names,"../client/query/rand")


# Experiment 3

In [6]:
def args_exp3(arg,graph):
    final = "server_exit"
    save_dir = "../client/query/exp3/"
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    
    current = arg*100
    current = current +[final]
    with open(os.path.join(save_dir,graph),"w") as file:
        file.write("\n".join(current))
args_exp3(['9 12 79920 1 1'],'com-dblp')
args_exp3(['9 12 321 1 1'],"amazon_q")
args_exp3(['9 12 1452 1 1'],'orani')
args_exp3(['9 12 1127 1 1'],"ego-facebook")
args_exp3(['9 12 6876 1 1'],"government")

# Experiment 4 Check

In [ ]:
def args_exp3(arg,graph):
    final = "server_exit"
    save_dir = "../client/query/exp3/"
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    
    current = arg*100
    current = current +[final]
    with open(os.path.join(save_dir,graph),"w") as file:
        file.write("\n".join(current))
args_exp3(['9 12 79920 1 1'],'com-dblp')
args_exp3(['9 12 321 1 1'],"amazon_q")
args_exp3(['9 12 1452 1 1'],'orani')
args_exp3(['9 12 1127 1 1'],"ego-facebook")
args_exp3(['9 12 6876 1 1'],"government")

# Experiment 5

In [7]:
df = pd.read_csv("../../Experiments SCS - Graph Statistics.csv")

In [31]:
graphs = """com-dblp
ego-facebook
government
orani""".split("\n")

In [32]:
import networkx as nx
from collections import Counter
from itertools import islice

In [33]:
def read_graph_file(filepath):
    with open(filepath, 'r') as f:
        next(f)  # Skip the first line
        return [tuple(map(int, line.split())) for line in f]

In [34]:
def find_closest_vertices(core_values, degrees, target_value, num_vertices=5):
    sorted_vertices = sorted(core_values.items(), key=lambda x: abs(x[1] - target_value))
    closest = list(islice((v for v, _ in sorted_vertices), num_vertices))
    return (closest, 
            [degrees[v] for v in closest], 
            [core_values[v] for v in closest])

In [46]:
final = "server_exit"
save_dir = "../client/query/exp4/"
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)
for g in graphs:
    edges = read_graph_file(os.path.join("../../../../graphs",g))
    graph = nx.Graph(edges)
    core_values = nx.core_number(graph)
    degrees = dict(graph.degree())
    new_path = os.path.join(save_dir,g)
    if not os.path.isdir(new_path):
        os.mkdir(new_path)
    
    for i in [5,10,15,20]:
        closest,deg,cor = find_closest_vertices(core_values,degrees,i,10)
        qid = np.random.choice(closest)
        query = "9 12 "+str(qid)+" 1 1\n"
        query = query*100
        query+=final
        with open(os.path.join(new_path,str(i)+".txt"),'w') as file_:
            file_.write(query) 

# Experiment 6

In [4]:
import os

In [9]:
def args_exp6(qid,graph):
    final = "server_exit"
    save_dir = "../client/query/exp6/"
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    for j in ['l','h']:
        new_path = os.path.join(save_dir,j)
        if not os.path.isdir(new_path):
            os.mkdir(new_path)
        new_path = os.path.join(new_path,graph)
        if not os.path.isdir(new_path):
            os.mkdir(new_path)
        if j == "l":
            for i in [3,6,9,12,15,18]:
                query = str(i)+" 21 "+str(qid)+" 1 1\n"
                query = query*100
                query+=final
                with open(os.path.join(new_path,str(i)+".txt"),'w') as file_:
                    file_.write(query)
        else:
            for i in [6,9,12,15,18,21]:
                query = "3 "+str(i) +" "+str(qid)+" 1 1\n"
                query = query*100
                query+=final
                with open(os.path.join(new_path,str(i)+".txt"),'w') as file_:
                    file_.write(query)
            

In [12]:
args_exp6(1473,'ego-facebook')
args_exp6(5387,"government")
args_exp6(238357 ,"amazon_q")
args_exp6(1266,'orani')
args_exp6(5367,'com-dblp')

# Experiment 7

In [3]:
def args_exp7(arg,graph):
    final = "server_exit"
    save_dir = "../client/query/exp7/"
    current = arg*100
    current = current +[final]
    with open(os.path.join(save_dir,graph),"w") as file:
        file.write("\n".join(current))
args_exp7(['9 12 1127 1 1'],"ego-facebook")
args_exp7(['9 12 6876 1 1'],"government")
args_exp7(['9 12 321 1 1'],"amazon_q")
args_exp7(['9 12 1452 1 1'],'orani')
args_exp7(['9 12 79920 1 1'],'com-dblp')

# Experiment 8

In [4]:
def args_exp8(arg,graph):
    final = "server_exit"
    save_dir = "../client/query/exp8/"
    current = arg*100
    current = current +[final]
    with open(os.path.join(save_dir,graph),"w") as file:
        file.write("\n".join(current))
args_exp8(['9 12 1127 1 1'],"ego-facebook")
args_exp8(['9 12 6876 1 1'],"government")
args_exp8(['9 12 321 1 1'],"amazon_q")
args_exp8(['9 12 1452 1 1'],'orani')
args_exp8(['9 12 79920 1 1'],'com-dblp')

# Experiment 6

In [1]:
def args_exp5(qid,graph):
    final = "server_exit"
    
    save_dir = "../client/query/exp5/"
    save_dir = os.path.join(save_dir,graph)
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    for n1,n2 in zip([3,6,9,12,15,18,21],[6,9,12,15,18,21,24]):
        current = list(map(str,[n1,n2,qid,1,1]))
        current = " ".join(current)
        current = [current]*10
        current += [final]
        with open(os.path.join(save_dir,str(n1)+"_"+str(n2)+".txt"),"w") as file:
            file.write("\n".join(current))

In [53]:
args_exp5(1127,"ego-facebook")
args_exp5(321,"amazon_q")
args_exp5(6876,"government")
args_exp5(1452,'orani')
args_exp5(79920,'com-dblp')

# Experiment 9

In [5]:
def args_exp9(arg,graph):
    final = "server_exit"
    save_dir = "../client/query/exp9/"
    current = arg*100
    current = current +[final]
    with open(os.path.join(save_dir,graph),"w") as file:
        file.write("\n".join(current))
args_exp9(['9 12 1127 1 1'],"ego-facebook")
args_exp9(['9 12 6876 1 1'],"government")
args_exp9(['9 12 321 1 1'],"amazon_q")
args_exp9(['9 12 1452 1 1'],'orani')
args_exp9(['9 12 79920 1 1'],'com-dblp')